# Challenge: Who makes the best classifier?

1. Form teams.

2. Use the file [`hatespeech_text_label_vote_RESTRICTED_100K.csv`](hatespeech_text_label_vote_RESTRICTED_100K.csv) that you can find in the current folder. You can also download the data [here](https://www.dropbox.com/sh/4mapojr85a6sc76/AABYMkjLVG-HhueAgd0qM9kwa?dl=0).

3. The data contain one column with tweets (their text only) and a column with a label (*Abusive, Hateful, Normal, Spam*). Calculate some descriptives (e.g., distribution of labels; most frequent words per label etc.)

4. Create machine learning classifiers to (a) identify spam; and (b) identify abusive and/or hateful tweets.

The team with the best classification pipeline wins!



---

### Background info on the data

Find more information on this dataset here:
http://www.wikicfp.com/cfp/servlet/event.showcfp?eventid=99774&copyownerid=99078


Founta, Antigoni Maria, Constantinos Djouvas, Despoina Chatzakou, Ilias Leontiadis, Jeremy Blackburn, Gianluca Stringhini, Athena Vakali, Michael Sirivianos, and Nicolas Kourtellis. "*Large scale crowdsourcing and characterization of twitter abusive behavior.*" In Proceedings of the International AAAI Conference on Web and Social Media, 2018.